In [ ]:
from astropy.coordinates import SkyCoord
from astropy.io import fits
from sunpy.coordinates import frames
# from sunpy.coordinates import sun
from sunpy.map import Map
import astropy.units as u
import importlib
import inv_utils as iu
# import matplotlib.pyplot as plt
import numpy as np
import remap
import warnings
from sunpy.util.exceptions import SunpyMetadataWarning

# Ignore specific warning
# Ignore specific SunpyMetadataWarnings
warnings.filterwarnings("ignore", category=SunpyMetadataWarning, message="Could not parse unit string.*")

#### Conversion from HMI SHARP to HMI SHARP-CEA dataset

In [ ]:
# Setup the paths for HMI SHARP and SHARP-CEA datasets
# -----------------------------
data_dir             = '/mn/stornext/d9/data/avijeetp/1_Projects/33_SST/HMI/20200807/'
file_field          = data_dir + 'hmi.sharp_720s.7436.20200807_060000_TAI.field.fits'
file_inclination    = data_dir + 'hmi.sharp_720s.7436.20200807_060000_TAI.inclination.fits'
file_azimuth        = data_dir + 'hmi.sharp_720s.7436.20200807_060000_TAI.azimuth.fits'
file_bp_cea         = data_dir + 'hmi.sharp_cea_720s.7436.20200807_060000_TAI.Bp.fits'
file_br_cea         = data_dir + 'hmi.sharp_cea_720s.7436.20200807_060000_TAI.Br.fits'
file_bt_cea         = data_dir + 'hmi.sharp_cea_720s.7436.20200807_060000_TAI.Bt.fits'

# -----------------------------
# data_dir = '/mn/stornext/d9/data/avijeetp/1_Projects/33_SST/HMI/sharp_example/'
# file_field       = data_dir + 'hmi.sharp_720s.377.20110216_180000_TAI.field.fits'
# file_inclination = data_dir + 'hmi.sharp_720s.377.20110216_180000_TAI.inclination.fits'
# file_azimuth     = data_dir + 'hmi.sharp_720s.377.20110216_180000_TAI.azimuth.fits'
# file_bp_cea = data_dir + 'hmi.sharp_cea_720s.377.20110216_180000_TAI.Bp.fits'
# file_br_cea = data_dir + 'hmi.sharp_cea_720s.377.20110216_180000_TAI.Br.fits'
# file_bt_cea = data_dir + 'hmi.sharp_cea_720s.377.20110216_180000_TAI.Bt.fits'

In [ ]:
# Load the data
field, field_header = fits.getdata(file_field, header=True)  
inclination = fits.getdata(file_inclination)
azimuth = fits.getdata(file_azimuth)
blos = field * np.cos(inclination * np.pi / 180.0)

In [ ]:
# Create sunpy maps for the SHARP datasets
field_map = Map(file_field)   # B
inclination_map = Map(file_inclination)  # theta
azimuth_map = Map(file_azimuth)  # phi
blos_map = Map(blos, field_map.meta)

# Create sunpy maps for the SHARP-CEA datasets
br_map = Map(file_br_cea)  # Bz
bt_map = Map(file_bt_cea)  # -By
bp_map = Map(file_bp_cea)  # Bx

In [ ]:
# Plot the SHARP-CEA data to get a sense of the data
iu.plot_images([bp_map.data, -bt_map.data, br_map.data], title=['Bx', 'By','Bz'], fontsize=10, figsize=(16,3.5), cmap='seismic', grid_shape=(1, 3), fig_title='SHARP-CEA Data')

In [ ]:
# Plot the SHARP-CEA data on the solar disk and also project it to disc center for reference
remap.plot_map_on_grid(br_map, vmin_percentile=0.5, vmax_percentile=99.5, figsize=(6,6), project_dc=True)

In [ ]:
# Plot the Blos map from the sharp data (with 180 degree rotation, south pole at the top)
blos_map.peek()

In [ ]:
# To convert from the SHARP data to SHARP-CEA data, we need to remap the data. The first step is to get the WCS information.
# Note that NAXIS1 corresponds to the x-axis and NAXIS2 corresponds to the y-axis. So, shape[1] corresponds to the x-axis and shape[0] corresponds to the y-axis.
wcs_dict = remap.get_wcs_info(blos_map, verbose=True)

In [ ]:
importlib.reload(remap)

In [ ]:
# Remap the Blos data to CEA projection. The remap2cea function returns a sunpy map object. 
blos_cea = remap.remap2cea(wcs_dict, blos, debug=False)
# Plot the remapped Blos CEA data. Note the data is also corrected for the 180 degree rotation.
# The NAN values can be interpolated if needed by setting missing in the remap2cea function.
blos_cea.peek()

In [ ]:
# When we know the target WCS, we can also remap the SHARP-CEA data to the target WCS directly
# using the sunpy reproject_to function. This function returns a sunpy map object.
# The SHARP-CEA data is usually more cropped than the SHARP data to avoid the nan values at the edges.
blos_map.reproject_to(br_map.wcs).peek()

In [ ]:
# Print the shapes of the SHARP and SHARP-CEA datas. We see that we have slightly larger FOV in latitude.
# But we are preserving more of the original data in the SHARP-CEA data.
print(f'Blos SHARP shape     : {blos.shape}')
print(f'Br SHARP-CEA shape   : {br_map.data.shape}')
print(f'Blos SHARP-CEA shape : {blos_cea.data.shape}')

In [ ]:
# Convert the SHARP field, inclination and azimuth to Cartesian components. Note the sign convention
# Also note that the datasets have CROTA2=180, so south is at the top.

# Note that for the HMI coordinates the conversion is as follows:
# Bx = -B sin(theta) sin(phi)   <- Note the negative sign and sin(phi) instead of cos(phi)
# By = B sin(theta) cos(phi)    <- Note the positive sign and cos(phi) instead of sin(phi)
# Bz = B cos(theta)

field_z = field * np.cos(inclination * np.pi / 180.0) # Bz with 180 deg rotation
bhor = field * np.sin(inclination * np.pi / 180.0)
field_y = bhor * np.cos(azimuth * np.pi / 180.0) #   By with 180 deg rotation # Note that 
field_x = -bhor * np.sin(azimuth * np.pi / 180.0) # -Bx with 180 deg rotation

In [ ]:
# The remap.bvect2cea function remaps the SHARP data to CEA projection. The function returns the remapped Bx, By, Bz and the header.
# Note that the function was written for HMI in mind, so it always assumes the south pole is at the top and Bx is negative.
# delta_l=0.03
bx, by, bz, header = remap.bvec2cea(wcs_dict, field_x, field_y, field_z, debug=False, missing='nan')
bz_map = Map(bz, header)
bx_map = Map(bx, header)
by_map = Map(by, header)

In [ ]:
# To highlight that the bvec2cea function takes 180 degree flipped -Bx, By, Bz as input, we plot the input and output data.
iu.plot_images([field_x, field_y, field_z, bx_map.data, by_map.data, bz_map.data], title=['-Bx (180)', 'By (180)', 'Bz (180)', 'Bx', 'By', 'Bz'], grid_shape=(2, 3), cmap='seismic', figsize=(16, 7), aspect='auto', fig_title='Comparison of input (top) and output (bottom) data for the bvec2cea function')

In [ ]:
# We can compare the SHARP-CEA data with the remapped SHARP data. We see that the data is very similar ecxept for the edges, where the HMI SHARP-CEA data is cropped.
iu.plot_images([bp_map.data, -bt_map.data, br_map.data, bx_map.data, by_map.data, bz_map.data], title=['Br', '-Bt', 'Bp', 'Bx', 'By', 'Bz'], grid_shape=(2, 3), cmap='seismic', figsize=(16, 7), aspect='auto', fig_title='Comparison of HMI SHARP-CEA data (top) with ours (bottom)', cb_pad=0.18)

---

#### Cropping the HMI SHARP dataset and then converting it to SHARP-CEA dataset (For the 2020-08-07) dataset

In [ ]:
blos_map.peek()

In [ ]:
# We manually crop a region aroung the sunspot umbra and plot it. The region is defined by the coordinates x1, x2, y1, y2.
x1, x2 = -476, -444 # arcsec
y1, y2 = 260, 292   # arcsec
bottom_left = SkyCoord(x1 * u.arcsec, y1 * u.arcsec, frame=blos_map.coordinate_frame)
top_right = SkyCoord(x2 * u.arcsec, y2* u.arcsec, frame=blos_map.coordinate_frame)
blos_submap = blos_map.submap(bottom_left, top_right=top_right)
print(blos_submap.data.shape)
blos_submap.peek()

In [ ]:
# We then crop the same region in field, inclination and azimuth maps and plot them.
# Note that the WCS info in submaps is updated automatically.
field_submap = field_map.submap(bottom_left, top_right=top_right)
inclination_submap = inclination_map.submap(bottom_left, top_right=top_right)
azimuth_submap = azimuth_map.submap(bottom_left, top_right=top_right)

In [ ]:
# We can also crop the SHARP-CEA data to the same region and plot it.
br_submap = br_map.submap(bottom_left, top_right=top_right)
bt_submap = bt_map.submap(bottom_left, top_right=top_right)
bp_submap = bp_map.submap(bottom_left, top_right=top_right)
br_submap.peek()
# Note that the br_submap region is slightly larger than the blos_submap region. 
# This could be because of the rotation of the data, that is not accounted for in the submap function properly.

In [ ]:
# Remember that for the HMI coordinates the conversion is as follows:
# Bx = -B sin(theta) sin(phi)   <- Note the negative sign and sin(phi) instead of cos(phi)
# By =  B sin(theta) cos(phi)    <- Note the positive sign and cos(phi) instead of sin(phi)
# Bz =  B cos(theta)
field_x_crop = -field_submap.data * np.sin(inclination_submap.data * np.pi / 180.0) * np.sin(azimuth_submap.data * np.pi / 180.0)
field_y_crop =  field_submap.data * np.sin(inclination_submap.data * np.pi / 180.0) * np.cos(azimuth_submap.data * np.pi / 180.0) 
field_z_crop =  field_submap.data * np.cos(inclination_submap.data * np.pi / 180.0)

In [ ]:
# Get the WCS info for the cropped SHARP data
wcs_crop = remap.get_wcs_info(blos_submap, verbose=True)

In [ ]:
# We can simply remap any scalar component using the remap2cea function. The function returns a sunpy map object.
blos_crop_cea = remap.remap2cea(wcs_crop, blos_submap.data, debug=False)
blos_crop_cea.peek()

In [ ]:
# For the vector transformation, we use the bvec2cea function. The function returns the remapped Bx, By, Bz and the header.
bx_crop, by_crop, bz_crop, header_crop = remap.bvec2cea(wcs_crop, field_x_crop, field_y_crop, field_z_crop, debug=False, missing='nan')
bx_submap = Map(bx_crop, header_crop)
by_submap = Map(by_crop, header_crop)
bz_submap = Map(bz_crop, header_crop)

In [ ]:
# We can now compare the plots. Remember that the regions were slightly different due to the rotation of the data and the way submap function works.
iu.plot_images([bp_submap.data, -bt_submap.data, br_submap.data, bx_crop, by_crop, bz_crop], title=['Br', '-Bt', 'Bp', 'Bx', 'By', 'Bz'], grid_shape=(2, 3), cmap='seismic', figsize=(16, 7), aspect='auto', fig_title='Comparison of cropped HMI SHARP-CEA data (top) with ours (bottom)', cb_pad=0.18)

---